In [1]:
#Models
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder , StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier , HistGradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score, log_loss, make_scorer
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
from skopt import BayesSearchCV
from skopt.space import Real, Integer
from sklearn.metrics import confusion_matrix

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np

#pd.options.display.float_format = '{:,.6f}'.format


In [3]:
#Generate a single frame to test best model

path_1 = r"/home/tomas-mur/Documentos/Codigos/payment/datasets/raw/Junio/Detalle_Ratio_1019_20240621110301.txt"
path_2 = r"/home/tomas-mur/Documentos/Codigos/payment/datasets/raw/Junio/Detalle_Ratio_1021_20240621150203.txt"

df_1 = pd.read_csv(path_1 , sep = "|" )
df_2 = pd.read_csv(path_2 , sep = "|" )

df = pd.concat([df_1 , df_2],axis = 0)
del df_1
del df_2

/tmp/ipykernel_3897/1762383436.py:7: DtypeWarning: Columns (48,54,55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2 = pd.read_csv(path_2 , sep = "|" )


In [4]:
df_grouped = df.groupby("PERIODO_FACTURADO")["CLIENTE"].size().reset_index(name='count')
df_sorted = df_grouped.sort_values(by="PERIODO_FACTURADO", ascending=True)

In [5]:
#Elimino las columnas que estan en la posicion de 50 en adelante, son inutiles y no aportan valor, no pertenecen a lo que el segmento maneja sino a datos de orientados a B2C
df.drop(columns=df.columns[50:], inplace = True)

#Formateo algunas columnas relacionadas a fechas como formato manejable 
df['PERIODO_FACTURADO'] = df['PERIODO_FACTURADO'].astype(str)
df['PERIODO_VENCIMIENTO'] = df['PERIODO_VENCIMIENTO'].astype(str)
df["CICLO"] = df["CICLO"].astype(str).str.replace(".0" , "",regex=True)

# Convertir PERIODO_FACTURADO a principio de mes
df['PERIODO_FACTURADO'] = pd.to_datetime(df['PERIODO_FACTURADO'] + '01', format='%Y%m%d') 

# Convertir PERIODO_VENCIMIENTO a fin de mes
df['PERIODO_VENCIMIENTO'] = pd.to_datetime(df['PERIODO_VENCIMIENTO'] + '01', format='%Y%m%d') + pd.offsets.MonthEnd(0)

#df["CICLO"] = pd.to_datetime(df["CICLO"] , format="%Y%m%d",coerce = True)

df["PERIODO_CONTRATO"] = (df['PERIODO_VENCIMIENTO']-df['PERIODO_FACTURADO']).dt.days


#Formateo la columna cliente para que sea legible
df["CLIENTE"] = df["CLIENTE"].astype(str).str.split('-').str[0]

df['PERIODO_FACTURADO'] = df['PERIODO_FACTURADO'].dt.date
df['PERIODO_VENCIMIENTO'] = df['PERIODO_VENCIMIENTO'].dt.date

# Elimino las variables que no aportan dado que fueron creadas principalmente para el segmento B2C
drop_cols = ["CICLO","CARTERA_CANALES","CARRIERS","INTRAGRUPO","SEGMENTO","SEGMENTO_AGRUPADO","RIESGO_ORIGINACION","RIESGO_CARTERA","FEC_ALTA","TIPO_DOCUMENTO","SEGMENTO_HOMOLOGADO"]
df.drop(columns=drop_cols , inplace = True)

# Ordeno por periodo
df = df.sort_values(by='PERIODO_FACTURADO', ascending=True)
periodos = df["PERIODO_FACTURADO"].unique()


# Creacion de columna de desplazamiento para ver el valor anterior(1 y 2 meses) de S0.

df['S0_PREV'] = df.groupby(["CLIENTE"])['S0'].shift(1)
df['S0_PREV2'] = df.groupby(["CLIENTE"])['S0'].shift(2)
df['S0_PREV3'] = df.groupby(["CLIENTE"])['S0'].shift(3)

# Diferencial entre pagos previos, si crece o disminuyen
df['GAP_PM2M'] = df['S0_PREV'] - df['S0_PREV2']
df['GAP_PM2M2'] = df['S0_PREV2'] - df['S0_PREV3']
df['GAP_PM2M'] = df['GAP_PM2M'].fillna(0)
df['GAP_PM2M2'] = df['GAP_PM2M2'].fillna(0)

# Generacion de  media movil de 3 meses para la facturacion / pagos a S0

#df["S0_3EMA"] = df.groupby(["CLIENTE"])['FACTURA_REAL'].rolling(window=3).mean()
#df["S0_3EMA"] = df.groupby(["CLIENTE"])['S0'].rolling(window=3).mean()

df["FACT_3EMA"] = df.groupby("CLIENTE")['FACTURA_REAL'].transform(lambda x: x.rolling(window=3).mean())
df["S0_3EMA"] = df.groupby("CLIENTE")['S0'].transform(lambda x: x.rolling(window=3).mean())

# Determinacion si un cliente pago el mes anterior en S0
df['PAGO_UM'] = df.apply(lambda row: 1 if row['S0_PREV'] > 0  else 0, axis=1)

# Determinar si un cliente pagó el mes anterior y no pagó este mes
df['PAGO_UM_C'] = df.apply(lambda row: 1 if row['S0_PREV'] > 0 and row['S0'] == 0 else 0, axis=1)



# Dropeo la columna
#df = df.drop(columns="S0_PREV") lo comente el 22-08-2024 a modo de prueba

In [6]:
# Generacion de la variable objetivo, es decir, a predecir

# Creacion de columna de desplazamiento para ver el valor futuro de S0
df['S0_NEXT'] = df.groupby('CLIENTE')['S0'].shift(-2)

# Determinacion de si un cliente pagará el próximo mes en la semana 0
df['PAGO_PM_C'] = df['S0_NEXT'].apply(lambda x: 1 if x > 0 else 0)

# Drop de la columna temporal S0_NEXT
df = df.drop(columns=['S0_NEXT']) 

In [7]:
# Agrupo los pagos cada 4 semanas, a excepcion de pago a vencimiento (S0).

df["0S"] = (df["S0"]  ) 
df["4S"] = ( df["S1"] +df["S2"] +df["S3"] + df["S4"]  ) 
df["8S"] = (df["S5"] + df["S6"] +df["S7"] +df["S8"] ) 
df["12S"] = (df["S9"] + df["S10"] +df["S11"] +df["S12"] ) 
df["16S"] = (df["S13"] + df["S14"] +df["S15"] +df["S16"] ) 
df["20S"] = (df["S17"] + df["S18"] +df["S19"] +df["S20"] ) 
df["24S"] = (df["S21"] + df["S22"] +df["S23"] +df["S24"] ) 

# Genero una variable que sea proporcional al pago

PR_S0 = pd.DataFrame((df.groupby("CLIENTE")["S0"].sum() / df.groupby("CLIENTE")["FACTURA_REAL"].sum()), columns=["PR_S0"])

In [8]:
# saco las features que ya estan incluidas en las agrupaciones

drop_cols_S = ['S0', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10',
       'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20',
       'S21', 'S22', 'S23', 'S24']
df.drop(columns=drop_cols_S,inplace=True)

In [9]:
# Seteo a  Cliente como indice para facilitar el merge.

df = df.reset_index().set_index("CLIENTE")

In [10]:
# Agrupo segun antiguedad de clietne.

df["VIGENCIA"] = np.where (df["VIGENCIA"] == "Alta Antigua Mayor a 12 meses" , "Mayor a 12 meses" , df["VIGENCIA"])
df["VIGENCIA"] = np.where ( (df["VIGENCIA"] == "Alta Nueva hasta 3 meses") | (df["VIGENCIA"] == "Alta Nueva de 4 a 6 meses")  , "Menor o igual a 6 meses" , df["VIGENCIA"])
df["VIGENCIA"] = np.where ( (df["VIGENCIA"] == "Entre 6 y 12 meses")  , "Alta Nueva de 7 a 12 meses" , df["VIGENCIA"])

# soluciono inconsistencias en los segmentos de atencion

df["SEGMENTO_ATENCION"] = np.where (df["SEGMENTO_ATENCION"] == "EMPRESAS ","EMPRESAS",df["SEGMENTO_ATENCION"]  )
df["SEGMENTO_ATENCION"] = np.where (df["SEGMENTO_ATENCION"] == "Empresas","EMPRESAS",df["SEGMENTO_ATENCION"]  )

In [11]:
# Unir los DataFrames utilizando el índice.

df_combined = pd.merge(df, PR_S0, left_index=True, right_index=True, how='left')

In [12]:
# Genero una nueva base.

df_combined = df_combined.reset_index()
df_combined = df_combined.sort_values("PERIODO_VENCIMIENTO")

# Saco las columnas que no van a aportar para el train test validation
drop_comb = ["PREFIJO" , "index" ,"NUMERO_FACTURA"  , "PERIODO_VENCIMIENTO" , "DEPARTAMENTO", "NUM_IDENT","CLIENTE","OFERTA" ]
df_combined.drop(columns=drop_comb, inplace=True)

In [13]:
# Aplico ordinal encoder para el atributo "VIGENCIA" que representa la permanencia dentro de TEF de c/empresa

# Lista de categorías ordenadas de menor a mayor

orden_categorias = ["Menor o igual a 6 meses", "Alta Nueva de 7 a 12 meses", "Mayor a 12 meses"]

#Aplico conversion

ord_enc = OrdinalEncoder(categories=[orden_categorias])

# Aplicar el encoder a la columna

df_combined['VIGENCIA'] = ord_enc.fit_transform(df_combined[['VIGENCIA']])

In [14]:
##Segmento

# Initialize the encoder
oh_enc_seg = OneHotEncoder(drop='first', sparse_output=False)

# Perform the one-hot encoding
encoded_columns_segmento = oh_enc_seg.fit_transform(df_combined[['SEGMENTO_ATENCION']])

# Create a DataFrame with the encoded columns

encoded_df = pd.DataFrame(encoded_columns_segmento, columns=oh_enc_seg.get_feature_names_out(['SEGMENTO_ATENCION']))

# Concatenate the original DataFrame (excluding the original column) with the encoded DataFrame

df_combined = pd.concat([df_combined.drop(columns=['SEGMENTO_ATENCION']), encoded_df], axis=1)

In [15]:
##Base

oh_enc_base = OneHotEncoder(drop='first', sparse_output=False)
encoded_columns_base = oh_enc_base.fit_transform(df_combined[["BASE"]])
encoded_df = pd.DataFrame(encoded_columns_base , columns = oh_enc_base.get_feature_names_out(["BASE"]))
df_combined = pd.concat( [df_combined.drop(columns=["BASE"]) , encoded_df] , axis = 1)

In [16]:
df_combined = df_combined.fillna(0)

In [19]:
df_combined.columns

Index(['PERIODO_FACTURADO', 'VALOR_FACTURA', 'VALOR_NOTA', 'FACTURA_REAL',
       'VIGENCIA', 'PERIODO_CONTRATO', 'S0_PREV', 'S0_PREV2', 'S0_PREV3',
       'GAP_PM2M', 'GAP_PM2M2', 'FACT_3EMA', 'S0_3EMA', 'PAGO_UM', 'PAGO_UM_C',
       'PAGO_PM_C', '0S', '4S', '8S', '12S', '16S', '20S', '24S', 'PR_S0',
       'SEGMENTO_ATENCION_EMPRESAS', 'SEGMENTO_ATENCION_PYMES',
       'SEGMENTO_ATENCION_nan', 'BASE_FS FIJA'],
      dtype='object')

In [20]:
df_combined.drop(columns=["PERIODO_FACTURADO"] , inplace=True)

In [21]:
df_combined.shape

(2372747, 27)

In [22]:
y = df_combined ["PAGO_PM_C"]
X = df_combined.drop(columns="PAGO_PM_C")

In [23]:
print(X.columns)


Index(['VALOR_FACTURA', 'VALOR_NOTA', 'FACTURA_REAL', 'VIGENCIA',
       'PERIODO_CONTRATO', 'S0_PREV', 'S0_PREV2', 'S0_PREV3', 'GAP_PM2M',
       'GAP_PM2M2', 'FACT_3EMA', 'S0_3EMA', 'PAGO_UM', 'PAGO_UM_C', '0S', '4S',
       '8S', '12S', '16S', '20S', '24S', 'PR_S0', 'SEGMENTO_ATENCION_EMPRESAS',
       'SEGMENTO_ATENCION_PYMES', 'SEGMENTO_ATENCION_nan', 'BASE_FS FIJA'],
      dtype='object')


In [24]:
X_train , X_test , y_train , y_test = train_test_split(X , y ,train_size= 0.7)

In [25]:
----------

SyntaxError: invalid syntax (1776373828.py, line 1)

In [25]:
clf_dt = DecisionTreeClassifier(random_state=8)
# Definir el grid de parámetros para GridSearchCV
param_grid = {
    'criterion': ['gini'],
    'max_depth': [ 5 , 15 ,10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Inicializar GridSearchCV con validación cruzada (cv=5 por ejemplo)
grid_search = GridSearchCV(estimator=clf_dt, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Ajustar GridSearchCV a los datos de entrenamiento
grid_search.fit(X_train, y_train)


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=8), n_jobs=-1,
             param_grid={'criterion': ['gini'], 'max_depth': [5, 15, 10, 20],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]},
             scoring='accuracy')

In [26]:
# Mostrar los mejores parámetros encontrados
print(f"Mejores parámetros: {grid_search.best_params_}")

# Evaluar el mejor modelo en los datos de prueba
y_pred = grid_search.best_estimator_.predict(X_test)
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))

# Mostrar el mejor modelo (árbol de decisión entrenado)
best_model = grid_search.best_estimator_
print(f"\nModelo óptimo: {best_model}")

Mejores parámetros: {'criterion': 'gini', 'max_depth': 15, 'min_samples_leaf': 1, 'min_samples_split': 2}

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.85      0.86      0.86    470797
           1       0.72      0.70      0.71    241028

    accuracy                           0.81    711825
   macro avg       0.79      0.78      0.78    711825
weighted avg       0.81      0.81      0.81    711825


Modelo óptimo: DecisionTreeClassifier(max_depth=15, random_state=8)


In [27]:
# Inicializar el clasificador Random Forest
clf_rf = RandomForestClassifier(random_state=8, n_jobs=-1)

# Definir el grid de parámetros para GridSearchCV
param_grid = {
    'n_estimators': [5,6,7 ],  # Número de árboles
    'criterion': ['gini'],  # Función de impureza
    'max_depth': [ 6 ,8,10,12,15],  # Profundidad máxima del árbol
    'min_samples_split': [2, 5, 10],  # Mínimo de muestras para dividir un nodo
    'min_samples_leaf': [1, 2, 4]  # Mínimo de muestras en hojas
}

# Inicializar GridSearchCV con validación cruzada (cv=5)
grid_search_rf = GridSearchCV(estimator=clf_rf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Ajustar GridSearchCV a los datos de entrenamiento
grid_search_rf.fit(X_train, y_train)

/home/tomas-mur/Documentos/.venv/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


GridSearchCV(cv=5, estimator=RandomForestClassifier(n_jobs=-1, random_state=8),
             n_jobs=-1,
             param_grid={'criterion': ['gini'], 'max_depth': [6, 8, 10, 12, 15],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [5, 6, 7]},
             scoring='accuracy')

In [28]:
# Mostrar los mejores parámetros encontrados
print(f"Mejores parámetros: {grid_search_rf.best_params_}")

# Evaluar el mejor modelo en los datos de prueba
y_pred = grid_search_rf.best_estimator_.predict(X_test)
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))

# Mostrar el mejor modelo (árbol de decisión entrenado)
best_model = grid_search_rf.best_estimator_
print(f"\nModelo óptimo: {best_model}")

Mejores parámetros: {'criterion': 'gini', 'max_depth': 15, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 7}

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.85      0.86      0.86    470797
           1       0.72      0.71      0.72    241028

    accuracy                           0.81    711825
   macro avg       0.79      0.79      0.79    711825
weighted avg       0.81      0.81      0.81    711825


Modelo óptimo: RandomForestClassifier(max_depth=15, min_samples_leaf=2, min_samples_split=5,
                       n_estimators=7, n_jobs=-1, random_state=8)


In [30]:
clf_lgbm = lgb.LGBMClassifier(random_state=8)

param_grid = {
    "num_leaves" : [15,31,50],
    "max_depth" : [5,10,15,20],
    "learning_rate": [0.01 , 0.05 , 0.1],
    "n_estimators" : [100],
    "min_child_samples" : [5,10,15]    
    }

grid_search_lgbm = GridSearchCV(
    clf_lgbm,
    param_grid,
    scoring = "accuracy",
    cv = 5,
    n_jobs=-1
)

grid_search_lgbm.fit(X_train , y_train)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 449403, number of negative: 879334
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 449403, number of negative: 879335
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 449403, number of negative: 879334
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 449403, number of negative: 879334
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 449403, number of negative: 879334
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 449403, number of negative: 879335
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[L

GridSearchCV(cv=5, estimator=LGBMClassifier(random_state=8), n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': [5, 10, 15, 20],
                         'min_child_samples': [5, 10, 15],
                         'n_estimators': [100], 'num_leaves': [15, 31, 50]},
             scoring='accuracy')

In [31]:
# Mostrar los mejores parámetros encontrados
print(f"Mejores parámetros: {grid_search_lgbm.best_params_}")

# Evaluar el mejor modelo en los datos de prueba
y_pred = grid_search_lgbm.best_estimator_.predict(X_test)
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))

# Mostrar el mejor modelo (árbol de decisión entrenado)
best_model = grid_search_lgbm.best_estimator_
print(f"\nModelo óptimo: {best_model}")

Mejores parámetros: {'learning_rate': 0.1, 'max_depth': 20, 'min_child_samples': 5, 'n_estimators': 100, 'num_leaves': 50}

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.85      0.87      0.86    470797
           1       0.73      0.71      0.72    241028

    accuracy                           0.81    711825
   macro avg       0.79      0.79      0.79    711825
weighted avg       0.81      0.81      0.81    711825


Modelo óptimo: LGBMClassifier(max_depth=20, min_child_samples=5, num_leaves=50, random_state=8)


In [26]:


scaler = StandardScaler()

# Ajustar y transformar X_train
X_train_std = scaler.fit_transform(X_train)

# Usar la misma transformación en X_test
X_test_std = scaler.transform(X_test)


In [27]:

clf_lg = LogisticRegression(random_state=8)

# Definir el espacio de búsqueda de hiperparámetros
param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet', None],  # Tipo de regularización
    'C': [ 0.05 ,0.1, 1],  # Fuerza de regularización
    'solver': ['liblinear', 'saga'],  # Solvers compatibles con L1 y ElasticNet
    'l1_ratio': [0, 0.5, 1]  # Solo aplica si 'penalty' es 'elasticnet'
}


# Realizar GridSearchCV con validación cruzada
grid_search = GridSearchCV(clf_lg, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_std, y_train)


/home/tomas-mur/Documentos/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/home/tomas-mur/Documentos/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/home/tomas-mur/Documentos/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/home/tomas-mur/Documentos/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/home/tomas-mur/Documentos/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1196: UserWarning: l1_ratio parameter is only used when pena

GridSearchCV(cv=5, estimator=LogisticRegression(random_state=8), n_jobs=-1,
             param_grid={'C': [0.05, 0.1, 1], 'l1_ratio': [0, 0.5, 1],
                         'penalty': ['l1', 'l2', 'elasticnet', None],
                         'solver': ['liblinear', 'saga']},
             scoring='accuracy')

In [28]:
# Mostrar los mejores parámetros encontrados
print(f"Mejores parámetros: {grid_search.best_params_}")

# Evaluar el mejor modelo en los datos de prueba
y_pred = grid_search.best_estimator_.predict(X_test)
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))

# Mostrar el mejor modelo (árbol de decisión entrenado)
best_model = grid_search.best_estimator_
print(f"\nModelo óptimo: {best_model}")

/home/tomas-mur/Documentos/.venv/lib/python3.10/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


Mejores parámetros: {'C': 0.1, 'l1_ratio': 0, 'penalty': 'l2', 'solver': 'saga'}

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.96      0.10      0.17    471278
           1       0.36      0.99      0.53    240547

    accuracy                           0.40    711825
   macro avg       0.66      0.54      0.35    711825
weighted avg       0.76      0.40      0.29    711825


Modelo óptimo: LogisticRegression(C=0.1, l1_ratio=0, random_state=8, solver='saga')
